## IRC Technical Assessment
I chose to use PostgreSQL to complete this assessment.

In [1]:
import psycopg2


In [ ]:
conn = psycopg2.connect(dbname="irc", user="brianrennie", password="brianrennie")

cursor = conn.cursor()

In [ ]:

cursor.execute("SELECT * FROM accounts limit 5;")
accounts = cursor.fetchall()
accounts

[('0x387D3247D2A5536C1C666CFBB259659F', 'household', 'United States'),
 ('0xB9C58CE7117BC12ED7DDEF301F798AA7', 'household', 'United States'),
 ('0xB0A6B44F19054E3BBF96FAA980267AE5', 'household', 'United States'),
 ('0x312EF920F30C85CEA85D1946B33AF12F', 'household', 'United States'),
 ('0x8A1F32025B28D0B32E84CF37CE392E18', 'household', 'United States')]

In [ ]:
cursor.execute("SELECT DISTINCT(account_id) FROM donations;")
donations = cursor.fetchall()
donations

[('0x9FDF4F29B92D2A22A1224FFA11C7AE22',),
 ('0x434DF548D6E02B73417CFA1307C4EFF5',),
 ('0x3B474C57CE07011F38D0B5BE824ECD9A',),
 ('0xF1DB9E484135F119FD14ABB309DE31D1',),
 ('0x4B1EB1ADC791E10ECBEBD0F05E4C31E4',),
 ('0x556F03C7290792473BF5C497D18D82B2',),
 ('0x3496312FF9AF5D917624122B63458243',),
 ('0x09170F45A9068B89BFCA9A2DF163AAF4',),
 ('0xA2A95FDA80503DD3311DACC796B6803E',),
 ('0x590DF3C9C415860463A2221C4BA80D05',),
 ('0x800AEC7C80836ADA761438BBC744F7C9',),
 ('0x8A0A13700FFB6B4B95BC9CD2AB437C36',),
 ('0x388A599375A1986C780911E2FCC15A9C',),
 ('0xAF7C96683ED3958C454970C3C5EDDE5D',),
 ('0xD915420490ED6477122113CCFC195DE4',),
 ('0xC30FA221A58BA64A516529D332B73B38',),
 ('0xA649FB02ECD7DFB154053FB09191344F',),
 ('0x8F1C99AF7F7B89FE3D204DC3A36BCC29',),
 ('0x7D3B22ED583D8D79613725A41D36E045',),
 ('0x90FC471C3F3082D3C0060FD29C9E3519',),
 ('0xB5A70F0EAC0A44246322521CA53A5FBA',),
 ('0x716B9F1EF7F5686C7E4D477790C66E56',),
 ('0x5A4E972B60CF024BC5D681C38319F717',),
 ('0x917F1AD29D430943B068041135351

## 1. What are the top five organizations by revenue in 2021?

In [2]:
conn = psycopg2.connect(dbname="irc", user="brianrennie", password="brianrennie")

cursor = conn.cursor()

cursor.execute('''SELECT account_id,close_date, SUM(amount) as total_amount
                    FROM donations 
                    WHERE account_id in (
                                         SELECT account_id
                                             FROM accounts
                                             WHERE account_type = 'organization'
                                        )
                    AND is_revenue = 'True'
                    GROUP BY 1,2
                    HAVING EXTRACT(YEAR FROM close_date) = 2021  
                    ORDER BY total_amount DESC limit 5;''')
result = cursor.fetchall()
result

[('0xAB7B829B9A1721326A19A7131A2659CC',
  datetime.date(2021, 4, 20),
  '$5,150.00'),
 ('0xD34FE50ED1B96D1C9D2308CD86F13530',
  datetime.date(2021, 12, 22),
  '$2,000.00'),
 ('0xCDC0C56B05B4D5743761F0085F0C7884',
  datetime.date(2021, 9, 15),
  '$1,025.00'),
 ('0xD34FE50ED1B96D1C9D2308CD86F13530', datetime.date(2021, 5, 15), '$200.00'),
 ('0x9BA80C6C39CB9080B63F9A4E14E56174', datetime.date(2021, 9, 15), '$175.00')]

## 2. Create an account-level table with aggregate measures of revenue in past week, past month, past year, and all time.

In [ ]:
conn = psycopg2.connect(dbname="irc", user="brianrennie", password="brianrennie")

cursor = conn.cursor()

cursor.execute('''WITH last_week_revenue as
                  (
                  SELECT account_id, close_date, amount, SUM(amount) OVER (PARTITION BY account_id order by close_date) as total_week_revenue
                    FROM donations 
                    WHERE DATE_PART('week', close_date) = 41.0
                    AND is_revenue = 'True'
                    ),

                    total_week_revenue as
                    (
                    SELECT account_id, MAX(lwr.total_week_revenue) as last_week_revenue FROM last_week_revenue lwr group by 1
                    ),

                    last_month_revenue as
                    (
                      SELECT account_id, close_date, amount, SUM(amount) OVER (partition by account_id ORDER BY close_date) as total_month_revenue
                    FROM donations
                    WHERE DATE_PART('month', close_date) = 9
                    AND is_revenue = 'True'
                    ),

                    total_month_revenue as
                    (
                    SELECT account_id, MAX(lmr.total_month_revenue) last_month_revenue FROM last_month_revenue lmr group by 1
                    ),

                    last_year_revenue as
                    (
                      SELECT account_id, close_date, amount, SUM(amount) OVER (partition by account_id ORDER BY close_date) as total_year_revenue
                    FROM donations
                    WHERE DATE_PART('year', close_date) = 2021
                    AND is_revenue = 'True'
                    ),

                    total_year_revenue as
                    (
                    SELECT account_id, MAX(lyr.total_year_revenue) last_year_revenue FROM last_year_revenue lyr group by 1
                    ),

                    all_time_revenue as
                    (
                      SELECT account_id, close_date, amount, SUM(amount) OVER (partition by account_id ORDER BY close_date) as all_time_revenue
                    FROM donations
                    WHERE is_revenue = 'True'
                    
                    ),

                    total_all_time_revenue as
                    (
                    SELECT account_id, MAX(atr.all_time_revenue) all_time_revenue FROM all_time_revenue atr group by 1
                    )

                    SELECT atr.account_id, 
                           last_week_revenue, 
                           last_month_revenue, 
                           last_year_revenue, 
                           all_time_revenue  
                    FROM total_all_time_revenue atr
                    LEFT JOIN total_week_revenue twr 
                    ON twr.account_id = atr.account_id
                    JOIN total_month_revenue tmr
                    ON  tmr.account_id = atr.account_id
                    JOIN total_year_revenue tyr
                    ON tyr.account_id = atr.account_id



                    
''')
result = cursor.fetchall()
result

[('0x00CD8F84803F8A1FE0BEA3DA900EF1D9', None, '$5.00', '$10.00', '$90.60'),
 ('0x03029B2287AACEA1FB66095D747AF9E2', None, '$20.00', '$20.00', '$80.00'),
 ('0x041F457FA61CB23CBD78A55F061F041C', None, '$0.00', '$0.00', '$0.00'),
 ('0x10D26A4ABC3BF89E8AF47714DDDC4161', None, '$16.66', '$16.66', '$31.66'),
 ('0x115954F4704D54C15F7AFF6E580B9F1F', None, '$20.00', '$0.00', '$30.00'),
 ('0x122442CC12FFFB7A636E9097FBAB40BF', None, '$0.00', '$25.00', '$25.00'),
 ('0x12BCA6DE84BFE968491DD6A07B4C2723', None, '$10.00', '$40.00', '$40.00'),
 ('0x135D68D87AA41AB73DAD3C23D48768B5', None, '$10.00', '$110.00', '$110.00'),
 ('0x1AB75049CD92928966762A7D3F29A2FC', None, '$10.00', '$60.00', '$177.76'),
 ('0x2013E66142C74D25049E312A3F60517B',
  '$61.54',
  '$3.57',
  '$25.00',
  '$220.41'),
 ('0x27E93F05E19E29DC120E5F99CFCE804D',
  '$15.00',
  '$200.00',
  '$200.00',
  '$215.00'),
 ('0x28964827259B52D5324A9F7CAEDE0866', None, '$33.33', '$33.33', '$33.33'),
 ('0x30C2B5E950B3F94998182657A6A50BD2', None, '$25.7

## 3. What is the average first donation of new donors for each billing country?

In [6]:
conn = psycopg2.connect(dbname="irc", user="brianrennie", password="brianrennie")

cursor = conn.cursor()

cursor.execute('''with numbered_donations as
                (
                SELECT a.account_id as account,
                       billing_country,
                       close_date,
                       amount::numeric as amount,
                       ROW_NUMBER () OVER (PARTITION BY a.account_id ORDER BY close_date) as donation_number
                       FROM accounts a
                       JOIN donations d 
                       ON a.account_id = d.account_id
                       WHERE d.is_revenue = 'False'
                       
                       
                ),

                first_donations as 
                (
                SELECT account,
                       billing_country,
                       close_date,
                       amount,
                       donation_number
                FROM numbered_donations
                WHERE donation_number = 1
                )     
                
                
                SELECT billing_country, AVG(ROUND(amount,2)) from first_donations GROUP BY 1''')
result = cursor.fetchall()
result

[('Germany', Decimal('46.2875000000000000')),
 ('United Kingdom', Decimal('39.9500000000000000')),
 ('United States', Decimal('59.9778528528528529'))]